In [15]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

print("TensorFlow:", tf.__version__)
print("GPU:", tf.config.list_physical_devices("GPU"))


TensorFlow: 2.10.0
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [16]:
TRAIN_DIR = Path(r"D:\ECG_model\train")
VAL_DIR   = Path(r"D:\ECG_model\validation")
TEST_DIR  = Path(r"D:\ECG_model\test")

IMG_SIZE = (224, 224)
BATCH_SIZE = 16


In [17]:
train_set = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_set = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_set = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    label_mode="categorical",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

NUM_CLASSES = train_set.element_spec[1].shape[-1]
CLASS_NAMES = train_set.class_names

print("Classes:", CLASS_NAMES)


Found 3528 files belonging to 7 classes.
Found 336 files belonging to 7 classes.
Found 144 files belonging to 7 classes.
Classes: ['Atrial_fibrillation(AF)', 'LBBB', 'Normal', 'PAC', 'PVC', 'RBBB', 'STD']


In [18]:
norm = tf.keras.layers.Rescaling(1./255)

train_ds = train_set.map(lambda x,y: (norm(x), y))
val_ds   = val_set.map(lambda x,y: (norm(x), y))
test_ds  = test_set.map(lambda x,y: (norm(x), y))

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.prefetch(tf.data.AUTOTUNE)
test_ds  = test_ds.prefetch(tf.data.AUTOTUNE)


In [19]:
def focal_loss(gamma=2.0, alpha=0.25):
    def loss_fn(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma)
        return tf.reduce_sum(weight * cross_entropy, axis=1)
    return loss_fn


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

for layer in base_model.layers:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
outputs = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(base_model.input, outputs)

model.compile(
    optimizer=Adam(learning_rate=1e-6),
    loss=focal_loss(gamma=2.0, alpha=0.25),
    metrics=["accuracy"]
)

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_4 (Rescaling)        (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 224, 224, 3)  7          ['rescaling_4[0][0]']            
 )                                                                                                
                                                                                            

In [21]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "efficientnet_final.h5",
        monitor="val_loss",
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=15,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]


In [ ]:
_ = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=80,     # upper limit, early stopping will stop earlier
    callbacks=callbacks
)
print("Done")

Epoch 1/80
  1/221 [..............................] - ETA: 2:04 - loss: 0.3508 - accuracy: 0.1875

ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/top_bn/FusedBatchNormV3' defined at (most recent call last):
    File "c:\Users\DELL\miniconda3\envs\tensorflow_env\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\DELL\miniconda3\envs\tensorflow_env\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\DELL\miniconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\DELL\miniconda3\envs\tensorflow_env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\DELL\miniconda3\envs\tensorflow_env\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\DELL\miniconda3\envs\tensorflow_env\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_l